# Soil
begin with good earth

In [ ]:
#| default_exp water/soil

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple, deque
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# unique
from treelib import Tree
import heapq


In [ ]:
#| export


from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, StyleDemo,NamedColor
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground,windy_edge, HexRegion, unique_windy_edge
from HexMagic.terrain import  TerraDemo, Terrain
from HexMagic.terrainpatterns import TerrainPatterns


In [ ]:
#| export
from HexMagic.voronoi import PlateKind
from HexMagic.weather import TerraDemo

## Generate

## Graphics

In [ ]:
#| export
@patch
def flow_directions(self: Terrain) -> np.ndarray:
    """Return array where each hex points to its downhill neighbor, or -1 if minimum."""
    n = len(self.elevations)
    directions = np.full(n, -1, dtype=int)
    
    for i in range(n):
        lowest = self.lowest_neighbor(i)
        if lowest is not None and self.elevations[lowest] < self.elevations[i]:
            directions[i] = lowest
    
    return directions

In [ ]:
#| export
@patch
def flow_diagram(self: Terrain) -> str:
    """Return array where each hex points to its downhill neighbor, or -1 if minimum."""
    n = len(self.elevations)
    overlay = ""
    style = StyleCSS("arrow", stroke="black",stroke_width=1)
    self.builder.add_style(style)

    directions = self.flow_directions()
    for i, x in enumerate(directions):
        if 0 <= x < n and self.elevations[i] > 0:
            overlay += self.hexGrid.arrow(i, x, style=style,fromMiddle=True,factor=0.1) + "\n"
    
    return overlay

In [ ]:
@patch
def mauiFlowDemo(self:TerraDemo, scale=0.5, debug=True):
    # Load California with geographic bounds
    demo = TerraDemo()
    terrain = demo.maui_map()

    # Check that coordinates were computed
    if debug:
        print(f"Latitude range: {terrain.fields['latitude'].min():.2f}° to {terrain.fields['latitude'].max():.2f}°")
        print(f"Longitude range: {terrain.fields['longitude'].min():.2f}° to {terrain.fields['longitude'].max():.2f}°")

    terrain.compute_weather(debug = debug)
    smaller = terrain.shrinkWeather(scale)
    smaller.hexGrid.builder.adjust("flow_diagram",smaller.flow_diagram())
    #smaller.hexGrid.update()

    #smaller.add_rain_overlay()
    return smaller.hexGrid.builder.show()

In [ ]:

TerraDemo().mauiFlowDemo()

Latitude range: 20.57° to 21.03°
Longitude range: -156.69° to -155.97°

Computing precipitation...
Grid: 100 x 70
Resolution: dx=1069m (~1.1km), dy=511m (~0.5km)
Latitude: 20.57° to 21.03° (center: 20.8°)
Wind: 8.0 m/s from 50.0°
Background precip: 0.15 mm/h

Precipitation range: 4 - 15267 mm/year
Mean: 2466 mm/year
Computing distance to coast...
Computing temperature...


# Soil

In [ ]:
#| export
@dataclass
class SoilType:
    """Represents a soil/bedrock type with erosion properties."""
    name: str
    bedrock: str  # Description of rock type
    erosion_resistance: float  # 0-1, higher = harder to erode
    permeability: float  # 0-1, higher = water flows through faster
    color: str  # Hex color for visualization
    
    @classmethod
    def standard_types(cls) -> List['SoilType']:
        """Five standard soil types from hard rock to sediment."""
        return [
            cls(
                name="Granite",
                bedrock="Plutonic igneous (continental crust)",
                erosion_resistance=0.9,
                permeability=0.2,
                color="#8B7355"  # Gray-brown
            ),
            cls(
                name="Basalt",
                bedrock="Volcanic igneous (oceanic crust)",
                erosion_resistance=0.85,
                permeability=0.3,
                color="#4A4A4A"  # Dark gray
            ),
            cls(
                name="Limestone",
                bedrock="Sedimentary carbonate (dissolves)",
                erosion_resistance=0.5,
                permeability=0.6,
                color="#D4C5B9"  # Light tan
            ),
            cls(
                name="Sandstone",
                bedrock="Sedimentary clastic (crumbles)",
                erosion_resistance=0.4,
                permeability=0.7,
                color="#C2B280"  # Sandy tan
            ),
            cls(
                name="Alluvial",
                bedrock="Deposited sediment (clay/silt)",
                erosion_resistance=0.1,
                permeability=0.4,
                color="#8B6914"  # Dark gold/brown
            ),
        ]

In [ ]:
#| export
@patch
def to_nc(self: SoilType) -> NamedColor:
    return NamedColor(self.color,self.name)
   


In [ ]:
#| export
@dataclass
class SoilSystem:
    """Manages soil types and their distribution across terrain."""
    terrain: 'Terrain'
    types: List[SoilType]
    regions: List[HexRegion]  # One per type (index matches types)
    
    def __post_init__(self):
        """Ensure terrain has soil_type field."""
        if 'soil_type' not in self.terrain.fields:
            self.terrain.fields['soil_type'] = np.zeros(len(self.terrain.elevations), dtype=int)

    @classmethod
    def from_plates(cls, terrain: 'Terrain', plates: List['Plate'], 
                    elev_threshold: float = 50.0,
                    precip_threshold: float = 750.0,
                    debug: bool = False) -> 'SoilSystem':
        """Create soil system from plate tectonics and terrain properties.
        
        Args:
            terrain: Terrain with elevations and climate data
            plates: List of tectonic plates
            elev_threshold: Elevation below which alluvial can form (meters)
            precip_threshold: Precipitation above which limestone forms (mm/year)
            debug: Print diagnostic info
        
        Returns:
            SoilSystem with initial soil distribution
        """

        
        types = SoilType.standard_types()
        n_hexes = len(terrain.elevations)
        
        # Initialize soil_type array
        soil_type = np.zeros(n_hexes, dtype=int)
        
        # Map plates to hexes
        plate_map = {}  # hex_idx -> plate
        for plate in plates:
            for hex_idx in plate.hexes:
                plate_map[hex_idx] = plate
        
        # Ensure we have distance_to_coast
        if 'distance_to_coast' not in terrain.fields:
            terrain.compute_distance_from_coast()
        
        # Track counts for debug
        counts = {i: 0 for i in range(len(types))}
        
        for i in range(n_hexes):
            elev = terrain.elevations[i]
            
            # Ocean hexes get basalt (oceanic crust)
            if elev <= 0:
                soil_type[i] = 1  # Basalt
                counts[1] += 1
                continue
            
            # Get plate type
            plate = plate_map.get(i)
            is_oceanic = plate and plate.kind == PlateKind.oceanic
            
            # Coastal lowlands → Alluvial
            if elev < elev_threshold and terrain.fields['distance_to_coast'][i] <= 1:
                soil_type[i] = 4  # Alluvial
                counts[4] += 1
            
            # High elevation → Bedrock (Granite or Basalt from plate)
            elif elev > 1000:
                if is_oceanic:
                    soil_type[i] = 1  # Basalt
                    counts[1] += 1
                else:
                    soil_type[i] = 0  # Granite
                    counts[0] += 1
            
            # Mid elevation → Sedimentary (Limestone or Sandstone)
            else:
                # Use precipitation if available
                if 'precipitation' in terrain.fields:
                    precip = terrain.fields['precipitation'][i]
                    if precip > precip_threshold:
                        soil_type[i] = 2  # Limestone (wet)
                        counts[2] += 1
                    else:
                        soil_type[i] = 3  # Sandstone (dry)
                        counts[3] += 1
                else:
                    # Fallback: use plate type
                    if is_oceanic:
                        soil_type[i] = 2  # Limestone
                        counts[2] += 1
                    else:
                        soil_type[i] = 3  # Sandstone
                        counts[3] += 1
        
        # Store in terrain
        terrain.fields['soil_type'] = soil_type
        
        # Create regions for each type
        regions = []
        for type_idx in range(len(types)):
            hexes = set(np.where(soil_type == type_idx)[0])
            regions.append(HexRegion(hexes=hexes, hexGrid=terrain.hexGrid))
        
        if debug:
            print("\n=== SOIL SYSTEM CREATED ===")
            print(f"Total hexes: {n_hexes}")
            for i, soil_type_obj in enumerate(types):
                pct = 100 * counts[i] / n_hexes
                print(f"{soil_type_obj.name:12s}: {counts[i]:5d} hexes ({pct:5.1f}%)")
        
        return cls(terrain=terrain, types=types, regions=regions)

### Soil overlay

In [ ]:
#| export
@patch
def soilOverlay(self:SoilSystem,f=None,smooth=False)->str:
    """ build an overlay simalar to HexGrid.styleLayer but uses plates."""
    
    aRender = self.terrain.hexGrid.builder
    sGrid = self.terrain.hexGrid
    terrain = self.terrain
    
    patGen = TerrainPatterns(self.terrain)
    cols = [x.to_nc() for x in SoilType.standard_types()]

    terrain.colorMap()
    
    # Find ocean hexes (level 0)
    ocean_hexes = terrain.find_region_at_level(0)
    ocean_region = HexRegion(hexes=ocean_hexes, hexGrid=sGrid)

    wave = patGen.wavePattern("ocean_waves_pat", 
                              amplitude=4, 
                              wavelength=16, 
                              color="#1565C0",      # stroke: medium blue
                              fill="#E3F2FD")       # fill: light blue
    oceanStyle = StyleCSS("ocean", fill=f"url(#ocean_waves_pat)")
    aRender.add_definition(wave)
    aRender.add_style(oceanStyle)


    patterns, soilStyles = patGen.namedHatchPattern(cols,stroke_width=4,spacing=8)
    
    # Add patterns to builder
    for p in patterns:
        aRender.add_definition(p)

    for s in soilStyles:
        aRender.add_style(s)

    for i, region in enumerate(self.regions):
        style = soilStyles[i]
        for h in region:
            sGrid.hexes[h].style = style

    for i in ocean_region:
        sGrid.hexes[i].style = oceanStyle
     
    ret = sGrid.styleLayerOrdered(
        styles=soilStyles,
        f=f)

    return ret

In [ ]:
@patch
def mauiSoilDemo(self:TerraDemo, scale=0.5, debug=True):
    # Load California with geographic bounds
    demo = TerraDemo()
    terrain = demo.maui_map()

    # Check that coordinates were computed
    if debug:
        print(f"Latitude range: {terrain.fields['latitude'].min():.2f}° to {terrain.fields['latitude'].max():.2f}°")
        print(f"Longitude range: {terrain.fields['longitude'].min():.2f}° to {terrain.fields['longitude'].max():.2f}°")

    terrain.compute_weather(debug = debug)
    smaller = terrain.shrinkWeather(scale)
    sampleSoil = SoilSystem.from_plates(smaller, [], debug=True)
    sGrid = smaller.hexGrid
    sGrid.builder.adjust("soilInformation", sampleSoil.soilOverlay(f=unique_windy_edge(iterations=2)))
    #smaller.hexGrid.update()

    #smaller.add_rain_overlay()
    return smaller.hexGrid.builder.show()

In [ ]:
TerraDemo().mauiSoilDemo()

Latitude range: 20.57° to 21.03°
Longitude range: -156.69° to -155.97°

Computing precipitation...
Grid: 100 x 70
Resolution: dx=1069m (~1.1km), dy=511m (~0.5km)
Latitude: 20.57° to 21.03° (center: 20.8°)
Wind: 8.0 m/s from 50.0°
Background precip: 0.15 mm/h

Precipitation range: 4 - 15267 mm/year
Mean: 2466 mm/year
Computing distance to coast...
Computing temperature...



=== SOIL SYSTEM CREATED ===
Total hexes: 1750
Granite     :   232 hexes ( 13.3%)
Basalt      :   900 hexes ( 51.4%)
Limestone   :   325 hexes ( 18.6%)
Sandstone   :   227 hexes ( 13.0%)
Alluvial    :    66 hexes (  3.8%)
